# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 9:07AM,247228,19,8575332,VITABIOTICS USA INC.,Released
1,Sep 26 2022 8:55AM,247225,10,0086006145,ISDIN Corporation,Released
2,Sep 26 2022 8:55AM,247225,10,0086006146,ISDIN Corporation,Released
3,Sep 26 2022 8:55AM,247225,10,0086006150,ISDIN Corporation,Released
4,Sep 26 2022 8:55AM,247225,10,0086006151,ISDIN Corporation,Released
5,Sep 26 2022 8:55AM,247225,10,0086006149,ISDIN Corporation,Released
6,Sep 26 2022 8:55AM,247225,10,0086006148,ISDIN Corporation,Released
7,Sep 26 2022 8:55AM,247225,10,0086006152,ISDIN Corporation,Released
8,Sep 26 2022 8:55AM,247225,10,0086006155,ISDIN Corporation,Released
9,Sep 26 2022 8:55AM,247225,10,0086006157,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,247222,Released,28
9,247223,Released,9
10,247224,Released,60
11,247225,Released,43
12,247228,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247222,NaN,28.0
247223,NaN,9.0
247224,NaN,60.0
247225,NaN,43.0
247228,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
247206,0.0,12.0
247210,5.0,1.0
247211,0.0,1.0
247215,0.0,1.0
247217,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
247206,0,12
247210,5,1
247211,0,1
247215,0,1
247217,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,247206,0,12
1,247210,5,1
2,247211,0,1
3,247215,0,1
4,247217,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,247206,,12
1,247210,5,1
2,247211,,1
3,247215,,1
4,247217,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 9:07AM,247228,19,VITABIOTICS USA INC.
1,Sep 26 2022 8:55AM,247225,10,ISDIN Corporation
44,Sep 26 2022 8:55AM,247224,10,ISDIN Corporation
104,Sep 26 2022 8:51AM,247222,10,ISDIN Corporation
132,Sep 26 2022 8:46AM,247223,10,Bio-PRF
141,Sep 26 2022 8:41AM,247220,10,Bio-PRF
145,Sep 26 2022 8:40AM,247219,10,ISDIN Corporation
160,Sep 26 2022 8:31AM,247217,10,Acute Outpatient Solutions
161,Sep 26 2022 8:27AM,247215,10,Acute Outpatient Solutions
162,Sep 26 2022 8:04AM,247211,10,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 26 2022 9:07AM,247228,19,VITABIOTICS USA INC.,,1
1,Sep 26 2022 8:55AM,247225,10,ISDIN Corporation,,43
2,Sep 26 2022 8:55AM,247224,10,ISDIN Corporation,,60
3,Sep 26 2022 8:51AM,247222,10,ISDIN Corporation,,28
4,Sep 26 2022 8:46AM,247223,10,Bio-PRF,,9
5,Sep 26 2022 8:41AM,247220,10,Bio-PRF,,4
6,Sep 26 2022 8:40AM,247219,10,ISDIN Corporation,,15
7,Sep 26 2022 8:31AM,247217,10,Acute Outpatient Solutions,,1
8,Sep 26 2022 8:27AM,247215,10,Acute Outpatient Solutions,,1
9,Sep 26 2022 8:04AM,247211,10,MedStone Pharma LLC,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 9:07AM,247228,19,VITABIOTICS USA INC.,1,
1,Sep 26 2022 8:55AM,247225,10,ISDIN Corporation,43,
2,Sep 26 2022 8:55AM,247224,10,ISDIN Corporation,60,
3,Sep 26 2022 8:51AM,247222,10,ISDIN Corporation,28,
4,Sep 26 2022 8:46AM,247223,10,Bio-PRF,9,
5,Sep 26 2022 8:41AM,247220,10,Bio-PRF,4,
6,Sep 26 2022 8:40AM,247219,10,ISDIN Corporation,15,
7,Sep 26 2022 8:31AM,247217,10,Acute Outpatient Solutions,1,
8,Sep 26 2022 8:27AM,247215,10,Acute Outpatient Solutions,1,
9,Sep 26 2022 8:04AM,247211,10,MedStone Pharma LLC,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 9:07AM,247228,19,VITABIOTICS USA INC.,1,
1,Sep 26 2022 8:55AM,247225,10,ISDIN Corporation,43,
2,Sep 26 2022 8:55AM,247224,10,ISDIN Corporation,60,
3,Sep 26 2022 8:51AM,247222,10,ISDIN Corporation,28,
4,Sep 26 2022 8:46AM,247223,10,Bio-PRF,9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 9:07AM,247228,19,VITABIOTICS USA INC.,1,NaN
1,Sep 26 2022 8:55AM,247225,10,ISDIN Corporation,43,NaN
2,Sep 26 2022 8:55AM,247224,10,ISDIN Corporation,60,NaN
3,Sep 26 2022 8:51AM,247222,10,ISDIN Corporation,28,NaN
4,Sep 26 2022 8:46AM,247223,10,Bio-PRF,9,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 9:07AM,247228,19,VITABIOTICS USA INC.,1,0.0
1,Sep 26 2022 8:55AM,247225,10,ISDIN Corporation,43,0.0
2,Sep 26 2022 8:55AM,247224,10,ISDIN Corporation,60,0.0
3,Sep 26 2022 8:51AM,247222,10,ISDIN Corporation,28,0.0
4,Sep 26 2022 8:46AM,247223,10,Bio-PRF,9,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2472186,163,5.0
19,494434,13,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2472186,163,5.0
1,19,494434,13,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,163,5.0
1,19,13,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,163.0
1,19,Released,13.0
2,10,Executing,5.0
3,19,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19
Status,,
Executing,5.0,0.0
Released,163.0,13.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19
0,Executing,5.0,0.0
1,Released,163.0,13.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19
0,Executing,5.0,0.0
1,Released,163.0,13.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()